<a href="https://colab.research.google.com/github/dgg32/crop_area/blob/main/Google_Earth_Engine_USA_Crops_Area.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Run me first

First of all, run the following cell to initialize the API. The output will contain instructions on how to grant this notebook access to Earth Engine using your account.

In [12]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=fNmvQhVY6QDM4q-TTeLRegCfZoPdQZjkBYoQlcY3Oe4&tc=odzzIRTPDY5XiPs096rXYo0R24Vs7B94VMjuP0TmqRs&cc=8HCwqpaT474MP48U6MvMdRZ-3BMALkeXjYgGxHPM40g

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1ARtbsJqN-RxtC25fimK0WakJXvYNTnb2MsENH8DKPvLKdtR2U5bPrLfJ1LA

Successfully saved authorization token.


In [79]:


areaPerPixel = ee.Image.pixelArea()


provinces = ee.FeatureCollection("FAO/GAUL/2015/level2");



def calculate_area (state, target_index, target, startDate, endDate):
    def generate_collection(feature):
      geometry = feature.geometry();
      
      dataset = ee.ImageCollection('USDA/NASS/CDL').filter(ee.Filter.date(startDate, endDate)).select("cropland").first()
                  
      o_dataset = dataset.clip(geometry);


      image = dataset.eq(target_index).selfMask().multiply(areaPerPixel).divide(1e6).set('system:time_start', startDate);
      
      
      reduce = image.select('cropland').reduceRegion(
                                                reducer= ee.Reducer.sum(),
                                                geometry= geometry,
                                                scale= 30,
                                                bestEffort= True
                                            );
            
      return feature.set(target, reduce.get("cropland"))

    filterLevel1 = ee.Filter.eq('ADM1_NAME', state);
    o_state = provinces.filter(filterLevel1);

    corn_areas = o_state.map(generate_collection)

    results = corn_areas.reduceColumns(ee.Reducer.sum(), [target])


    return results.values(["sum"]).get(0)

tasks = []

name_list = ee.ImageCollection('USDA/NASS/CDL').select("cropland").first().get("cropland_class_names").getInfo()

targets = ["Corn", "Cotton"]

state_list = ee.List(["Iowa", "Mississippi"])

for target in targets:

    for startDate in ['2015-01-01', '2016-01-01']:
        endDate = ee.Date(startDate).advance(1, "year")

        target_index = name_list.index(target) + 1;

        results = state_list.map(lambda state: calculate_area(state, target_index, target, startDate, endDate)).reduce(ee.Reducer.sum());


        task = ee.batch.Export.table.toDrive(collection=ee.FeatureCollection([ee.Feature(None, {"time": startDate, target: results})]),
                                        description=f'{target}_{startDate}_python_crop',
                                        folder='crop',
                                        fileFormat='CSV')
        task.start()

        tasks.append(task)


In [83]:
for task in tasks:
    print (task.status())

{'state': 'COMPLETED', 'description': 'Corn_2015-01-01_python_crop', 'creation_timestamp_ms': 1664185553153, 'update_timestamp_ms': 1664185562104, 'start_timestamp_ms': 1664185558006, 'task_type': 'EXPORT_FEATURES', 'destination_uris': ['https://drive.google.com/#folders/1Q5L2e0d_kbKjc63rMWaleJ35jJputNjr'], 'attempt': 1, 'batch_eecu_usage_seconds': 1.3087425231933594, 'id': 'PPEBEQWX2LJARGTFDKIZZPWO', 'name': 'projects/earthengine-legacy/operations/PPEBEQWX2LJARGTFDKIZZPWO'}
{'state': 'COMPLETED', 'description': 'Corn_2016-01-01_python_crop', 'creation_timestamp_ms': 1664185553466, 'update_timestamp_ms': 1664185584341, 'start_timestamp_ms': 1664185559091, 'task_type': 'EXPORT_FEATURES', 'destination_uris': ['https://drive.google.com/#folders/1Q5L2e0d_kbKjc63rMWaleJ35jJputNjr'], 'attempt': 1, 'batch_eecu_usage_seconds': 0.9537205696105957, 'id': '4HIZR4KMPC37VMPAVG2QJDOB', 'name': 'projects/earthengine-legacy/operations/4HIZR4KMPC37VMPAVG2QJDOB'}
{'state': 'COMPLETED', 'description': 'C